# 6.2.2 챗본 문답 데이터 감정 분류 모델 구현

In [1]:
!git clone https://github.com/songys/Chatbot_data

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 60 (delta 23), reused 5 (delta 3), pack-reused 18
Unpacking objects: 100% (60/60), 463.73 KiB | 4.64 MiB/s, done.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

### 데이터 읽어오기

In [3]:
train_file = '/content/Chatbot_data/ChatbotData.csv'
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

#chatbotdata 파일을 읽어와 CNN모델 학습시 필요한 Q(질문)와 label(감정) 데이터를 features와 labels 리스트에 저장함

In [4]:
# 단어 인덱스 시퀀스 벡터 2

corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

# 1에서 불러온 질문 리스트(features)에서 문장을 하나씩 꺼내와 text_to_word_sequence()함수를 이용해 단어 시퀀스를 만듦
# 단어 시퀀ㅅ스란 토큰들의 순차적 리스트를 의미함
# 예를 들어 '3박4일 놀러가고 싶다'면 ['3박4일', '놀러가고', '싶다']이렇게 된다.
# 위 처럼 생성된 단어 시퀀스를 말뭉치(corpus) 리스트에 저장함
# 다음 texts_to_sequences()함수를 이용해 문장 내 모든 단어를 시퀀스 번호(벡터)로 변환함

In [5]:
MAX_SEQ_LEN = 15 #단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

#벡터 크기가 재각각이여서 학습시키려면 같게 만들어줘야함
# max_seq_len을 이용해 벡터 크기 맞춤
# 크기가 작은 단어는 공간이 남는데 그 부분을 0으로 채워준다.
# 이 과정을 패딩 padding이라 한다. 
# 케라스에서 pad_sequences() 함수를 통해 시퀀스의 패딩 처리를 손쉽게 할 수 있다.
# maxlen 인자로 시퀀스의 최대 길이를 정하는데 학습시킬 문장 데이터들을 사전에 분석해 파악해야한다.
# 너무 크면 자원 낭비, 작게 잡으면 데이터 손실이 발생

In [6]:
#학습용, 검증용, 데이터셋 생성 3
#학습셋:검증셋:데이트셋 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

In [7]:
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

In [8]:
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [9]:
#하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE =128
epoch =5
vocab_size =len(word_index) + 1 #전체 단어 수 

In [10]:
#CNN 모델 정의 4
#단어 임베딩 처리 영역
input_layer = Input(shape = (MAX_SEQ_LEN,))
embedding_layer = Embedding(vocab_size, EMB_SIZE, input_length = MAX_SEQ_LEN)(input_layer)
# vocab_size = 단어 개수, EMB_SIZE= 임베딩 결과로 나올 밀집 벡터의 크기, input_length= 입력되는 시퀀스 벡터의 크기를 embedding()의 인자로 사용해 임베딩 계층을 생성
dropout_emb = Dropout(rate = dropout_prob)(embedding_layer) #dropout=0.5

#합성곱 필터와 연산을 통해 문장의 특징 정보를 추출하고, 평탄화를 하는 영역
conv1 = Conv1D(filters = 128,
               kernel_size = 3,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(filters = 128,
               kernel_size = 4,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(filters = 128,
               kernel_size = 5,
               padding = 'valid',
               activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3]) #concatenate는 특징맵 결과를 하나로 묶어줌

#fully connected layer
hidden = Dense(128, activation = tf.nn.relu)(concat)
dropout_hidden = Dropout(rate = dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden) #logits을 점수라 부름 출력 노드에 3개의 점수가 출력됨
predictions = Dense(3, activation=tf.nn.softmax)(logits)

In [11]:
# 모델 생성 5
model = Model(inputs=input_layer, outputs=predictions)

model.compile(optimizer ='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
#모델 학습하기 6
model.fit(train_ds, validation_data =val_ds, epochs=epoch, verbose=2)

Epoch 1/5
414/414 - 14s - loss: 0.8912 - accuracy: 0.5804 - val_loss: 0.5434 - val_accuracy: 0.8046 - 14s/epoch - 33ms/step
Epoch 2/5
414/414 - 3s - loss: 0.5017 - accuracy: 0.8157 - val_loss: 0.2763 - val_accuracy: 0.9179 - 3s/epoch - 6ms/step
Epoch 3/5
414/414 - 3s - loss: 0.3150 - accuracy: 0.8910 - val_loss: 0.1689 - val_accuracy: 0.9446 - 3s/epoch - 7ms/step
Epoch 4/5
414/414 - 4s - loss: 0.1851 - accuracy: 0.9375 - val_loss: 0.1095 - val_accuracy: 0.9674 - 4s/epoch - 10ms/step
Epoch 5/5
414/414 - 3s - loss: 0.1315 - accuracy: 0.9594 - val_loss: 0.0724 - val_accuracy: 0.9793 - 3s/epoch - 7ms/step


In [15]:
# 모델 평가 7 
loss, accuracy = model.evaluate(test_ds, verbose=2) # evaluate()함수를 이용해 성능을 평가함 
print('accuracy : %f' % (accuracy *100))
print('loss : %f' % (loss))

60/60 - 0s - loss: 0.0800 - accuracy: 0.9746 - 364ms/epoch - 6ms/step
accuracy : 97.461927
loss : 0.080039


In [16]:
model.save('cnn_model.h5') # 모델 저장 8